In [ ]:
# import zipfile
# zip_ref=zipfile.ZipFile('book-recommendation-dataset.zip')
# zip_ref.extractall()
# zip_ref.close()

In [26]:
import pandas as pd
books=pd.read_csv('Books.csv')
users=pd.read_csv('Users.csv')
ratings=pd.read_csv('Ratings.csv')

C:\Users\alok4\AppData\Local\Temp\ipykernel_25436\3026388437.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('Books.csv')


In [27]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [28]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [29]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [30]:
# Not Useful "Image UR-L", "Image UR-S", "Image UR-M", "Year of publication"
# The Image URLs will be useful for thumbnails on website

In [31]:
books['Image-URL-M'][1]

'http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

In [32]:
print(users.shape)
print(ratings.shape)
print(books.shape)

(278858, 3)
(1149780, 3)
(271360, 8)


In [33]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [34]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [35]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [36]:
books.duplicated().sum()

np.int64(0)

In [37]:
ratings.duplicated().sum()

np.int64(0)

In [38]:
users.duplicated().sum()

np.int64(0)

### Popularity Based Recommender System

In [39]:
ratings_with_name=ratings.merge(books,on="ISBN")

In [40]:
# num_rating_df=ratings_with_name.groupby("Book-Title").count()['Book-Rating'].reset_index()
# num_rating_df.rename(columns={"Book-Rating":'num_rating'},inplace=True)
# num_rating_df

In [46]:
avg_rating_df=ratings_with_name.groupby("Book-Title").corr()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={"Book-Rating":'avg_rating'},inplace=True)
avg_rating_df

ValueError: could not convert string to float: 'Karen Hesse'